In [ ]:
from collections import Counter
from pprint import pprint

from script.utils import load_indexed_manual_anaphoras

ana_count_ref = load_indexed_manual_anaphoras()

total = []
for x in ana_count_ref.values():
  total = total + x

unresolved = [x for x in total if x[2] == '?']
print('len(unresolved)', len(unresolved))
pos_counter = Counter()
for x in unresolved:
  pos_counter.update(str(x[0]))

pprint(pos_counter)


In [ ]:
from script.sbnutils import parse_research_data, extract_sbn_integration, extract_lambda_info
import os
import re
from collections import Counter, defaultdict
from pprint import pprint

from script.utils import to_abspath, load_indexed_manual_anaphoras, load_pmb_ids, load_spr_rt_with_subj_by_keys, \
  load_et_rt_with_subj_by_keys, load_et_with_subj_df

ANA_WORDS = ["he'd", 'himself.', 'herself', 'her.', "he'll", 'him.', 'my', 'she', 'they', 'his', 'itself.', 'him', 'Paul', 'me.', "they'll", 'her',
             'himself', 'them.', 'their', 'you', 'he', 'her,', 'that.', 'it', 'our', 'your', 'Charlie', 'it.', 'them']

print_acc = []


def wrap_print(*messages):
  joined_message = ' '.join(messages) + '\n'
  print_acc.append(joined_message.replace('\n\n', '\n'))
  print(messages)


def flush_clean_wrap_print(file_path):
  with open(file_path, 'w') as out:
    out.writelines(print_acc)
  print_acc.clear()


def extract_verbs_and_role_distances(non_connector_sbn_lines):
  global pmb_id, sent_nr
  roles_data = []
  sbn_verb_lines = [(idx, line) for idx, line in enumerate(non_connector_sbn_lines) if '.v.' in line.sbn_word]
  # TODO: sanity check if the integration matches the type of target
  for idx, verb_line in sbn_verb_lines:
    if verb_line.word_info.word == '':
      wrap_print(f'Skipping sbn verb line of {str(verb_line)}')
      continue
    wrap_print('-' * 3)
    wrap_print(f'{verb_line.sbn_word} - {verb_line.word_info.word} ({verb_line.word_info.word_with_context})')

    sent_idxs, sent_tokens = determine_ori_sent_token_idxs_based_on_sbn(sent_nr, verb_line.word_info.start_idx, verb_line.word_info.end_idx)
    if len(sent_tokens) > 1:
      clean_sent_tokens = [x.replace('.', '').replace(',', '') for x in sent_tokens]
      word_to_find = verb_line.word_info.word
      if ' ' in word_to_find:
        word_to_find = word_to_find.split(' ')[0]
      word_idx = clean_sent_tokens.index(word_to_find)
      word_pos = sent_idxs[word_idx] + 1
      target_token = word_to_find
    else:
      word_pos = sent_idxs[0] + 1
      target_token = sent_tokens[0]

    for role, role_value in verb_line.roles:
      if role_value.startswith('+'):
        target_idx = idx + int(role_value.replace('+', ''))
      elif role_value.startswith('-'):
        target_idx = idx - int(role_value.replace('-', ''))
      elif role_value.startswith('&gt;') or role_value.startswith('&lt;'):
        wrap_print(f'?{role} ({role_value}) -> ?')
        target_idx = -1
      elif role_value == 'speaker' or role_value == 'hearer':
        wrap_print(f'?{role} ({role_value}) -> meta({role_value})')
        target_idx = -1
      else:
        raise Exception(f'role_value misform: {role_value}')

      if target_idx != -1:
        target_sbn_line = non_connector_sbn_lines[target_idx]
        word_info_word = target_sbn_line.word_info.word if 'word' in target_sbn_line.word_info else 'None'
        wrap_print(f'{role} ({role_value}) -> {target_sbn_line.sbn_word} - {word_info_word} ({target_sbn_line.word_info.word_with_context})')

      roles_data.append(dict(
        sbn_verb_pos=idx,  # this is the index of this verb in sbn
        role=role,  # the role detected
        role_value=role_value,  # the value of the role / distance
        word_pos=word_pos,
        target_token=target_token,
        sent_idxs=sent_idxs,
        sent_tokens=sent_tokens,
      ))

  wrap_print('=' * 10)
  # TODO: collect word_pos for rt collection
  return roles_data


def get_sentence_tokens(pmb_id, sent):
  if pmb_id == 'd0349_p37':
    return [([0], 'Roger'), ([1], 'smiled'), ([2], 'and'), ([3, 4], 'sat~down.')]
  if pmb_id == 'd0349_p40':
    return [([0], 'Turn'), ([1], 'your'), ([2, 3], 'hearing~aid'), ([4], 'up.')]
  if pmb_id == 'd0349_p46':
    return [([0], 'Harry'), ([1], 'smiled'), ([2], 'at'), ([3], 'his'), ([4, 5], 'little~brother.')]
  if pmb_id == 'd0349_p70':
    return [([0], "I'll"), ([1], 'give'), ([2], 'him'), ([3, 4], 'more~than'), ([5], 'a'), ([6], 'slap.')]
  if pmb_id == 'd0349_p76':
    return [([0], 'Steve'), ([1, 2], 'sat~down'), ([3], 'to'), ([4], 'eat'), ([5], 'his'), ([6], 'lunch.')]
  if pmb_id == 'd0349_p87':
    return [([0], 'The'), ([1, 2], 'prison~guard'), ([3], 'walked'), ([4], 'along'), ([5], 'the'), ([6], 'row.')]
  if pmb_id == 'd0349_p94':
    return [([0], 'He'), ([1, 2], 'called~out'), ([3], 'but'), ([4], 'she'), ([5], 'did'), ([6], 'not'), ([7], 'move.')]
  if pmb_id == 'd0349_p95':
    return [([0], 'He'), ([1, 2], 'sat~down'), ([3], 'and'), ([4], 'began'), ([5], 'to'), ([6], 'eat'), ([7], 'breakfast.')]
  if pmb_id == 'd0350_p02':
    return [([0], 'William'), ([1, 2], 'sat~down'), ([3], 'and'), ([4], 'tucked'), ([5], 'into'), ([6], 'his'), ([7], 'breakfast.')]
  if pmb_id == 'd0350_p08':
    return [([0], 'Shut'), ([1], 'your'), ([2], 'smart'), ([3], 'foul'), ([4], 'mouth'), ([5], 'and'), ([6, 7], 'sit~down.')]
  if pmb_id == 'd0350_p19':
    return [([0], 'He'), ([1, 2], 'is~after'), ([3], 'glory'), ([4], 'and'), ([5], 'heads'), ([6], 'for'), ([7], 'his'), ([8], 'hall.')]
  if pmb_id == 'd0350_p25':
    return [([0], 'I'), ([1], 'was'), ([2, 3], 'thinking~of'), ([4], 'getting'), ([5], 'the'), ([6], 'band'), ([7], 'together'), ([8], 'again.')]
  if pmb_id == 'd0350_p39':
    return [([0], 'They'), ([1, 2], 'turned~round'), ([3], 'just'), ([4], 'as'), ([5], 'Frankie'), ([6], 'walked'), ([7], 'toward'), ([8], 'them.')]
  if pmb_id == 'd0350_p43':
    return [([0], 'All'), ([1], 'the'), ([2], 'bowls'), ([3], 'from'), ([4], 'the'), ([5], 'empty'), ([6], 'tables'), ([7], 'are'),
            ([8, 9], 'cleared~up.')]
  if pmb_id == 'd0350_p57':
    return [([0], 'Joseph'), ([1], 'walked'), ([2], 'into'), ([3], 'the'), ([4], 'office'), ([5], 'and'), ([6], 'removed'), ([7], 'his'),
            ([8, 9], 'flat~cap.')]
  if pmb_id == 'd0350_p73':
    return [([0], 'She'), ([1, 2], 'wound~up'), ([3], 'talking'), ([4], 'to'), ([5], 'Barry'), ([6], 'about'), ([7], 'her'), ([8], 'time'),
            ([9], 'in'), ([10], 'India.')]
  if pmb_id == 'd0350_p83':
    return [([0], 'Finally'), ([1], 'Maria'), ([2, 3], 'sat~down'), ([4], 'with'), ([5], 'a'), ([6], 'cup'), ([7], 'of'), ([8], 'tea'), ([9], 'and'),
            ([10], 'a'), ([11], 'sandwich.')]
  if pmb_id == 'd0350_p96':
    return [([0], 'It'), ([1], 'is'), ([2], 'not'), ([3], 'the'), ([4], 'most'), ([5], 'fragrant'), ([6], 'of'), ([7], 'places'), ([8], 'but'),
            ([9], 'we'), ([10, 11], 'make~do.')]
  if pmb_id == 'd0350_p97':
    return [([0], 'James'), ([1], 'walked'), ([2], 'in'), ([3], 'to'), ([4], 'wake'), ([5], 'Samuel'), ([6], 'at'), ([7, 8], 'four~thirty'),
            ([9], 'in'), ([10], 'the'), ([11], 'morning.')]
  if pmb_id == 'd0350_p98':
    return [([0], 'Let'), ([1], 'us'), ([2], 'give'), ([3], 'battle'), ([4], 'to'), ([5], 'these'), ([6], 'foreign'), ([7], 'snakes'),
            ([8, 9, 10, 11], 'once~and~for~all.')]
  if pmb_id == 'd0350_p99':
    return [([0], 'Tommy'), ([1], 'came'), ([2], 'out'), ([3], 'of'), ([4], 'the'), ([5], 'staff'), ([6], 'room'), ([7], 'toilet'), ([8], 'looking'),
            ([9, 10], 'a~little'), ([11], 'pale.')]
  if pmb_id == 'd0351_p22':
    return [([0], 'Bob'), ([1], 'sat'), ([2], 'at'), ([3], 'the'), ([4], 'entrance'), ([5], 'to'), ([6], 'the'), ([7], 'warehouse'), ([8], 'and'),
            ([9, 10], 'made~up'), ([11], 'a'), ([12], 'cigarette.')]
  if pmb_id == 'd0351_p24':
    return [([0], 'Brian'), ([1, 2], 'sat~down'), ([3], 'at'), ([4], 'an'), ([5], 'empty'), ([6], 'table'), ([7], 'and'), ([8], 'began'), ([9], 'to'),
            ([10], 'eat'), ([11], 'his'), ([12], 'breakfast.')]
  if pmb_id == 'd0351_p25':
    return [([0, 1, 2], 'One~by~one'), ([3], 'she'), ([4], 'placed'), ([5], 'her'), ([6], 'teeth'), ([7], 'about'), ([8], 'their'), ([9], 'necks'),
            ([10], 'and'), ([11], 'killed'), ([12], 'them.')]
  if pmb_id == 'd0351_p31':
    return [([0], 'The'), ([1, 2], 'police~officer'), ([3], 'got'), ([4], 'out'), ([5], 'of'), ([6], 'the'), ([7], 'van'), ([8], 'and'),
            ([9], 'picked'), ([10], 'up'), ([11], 'the'), ([12], 'two'), ([13], 'cans.')]
  if pmb_id == 'd0351_p52':
    return [([0], 'David'), ([1], 'came'), ([2], 'running'), ([3], 'into'), ([4], 'the'), ([5, 6], 'waiting~room'), ([7], 'and'), ([8], 'went'),
            ([9], 'straight'), ([10], 'to'), ([11], 'Stephen.')]
  if pmb_id == 'd0351_p58':
    return [([0], 'Arthur'), ([1], 'got'), ([2], 'in'), ([3], 'his'), ([4], 'truck'), ([5], 'and'), ([6], 'slammed'), ([7], 'his'), ([8], 'fist'),
            ([9], 'against'), ([10], 'the'), ([11, 12], 'steering~wheel.')]
  if pmb_id == 'd0351_p61':
    return [([0], 'Within'), ([1], 'four'), ([2], 'months'), ([3], 'they'), ([4], 'moved'), ([5], 'in'), ([6], 'together'), ([7], 'to'), ([8], 'the'),
            ([9], 'flat'), ([10], 'on'), ([11, 12], 'Shakespeare~Road.')]
  if pmb_id == 'd0351_p70':
    return [([0], 'Donald'), ([1], 'sat'), ([2], 'down'), ([3], 'at'), ([4], 'the'), ([5], 'kitchen'), ([6], 'table'), ([7], 'and'), ([8], 'fed'),
            ([9], 'his'), ([10, 11], 'little~sister'), ([12], 'her'), ([13], 'bottle.')]
  if pmb_id == 'd0351_p71':
    return [([0], 'Frank'), ([1], 'sighed'), ([2], 'and'), ([3], 'walked'), ([4], 'to'), ([5], 'the'), ([6, 7], 'visiting~room'), ([8], 'with'),
            ([9], 'his'), ([10], 'hands'), ([11], 'in'), ([12], 'his'), ([13], 'pockets.')]
  if pmb_id == 'd0352_p05':
    return [([0], 'She'), ([1], 'was'), ([2], 'served'), ([3], 'by'), ([4], 'an'), ([5], 'incredibly'), ([6], 'striking'), ([7], 'girl'),
            ([8], 'with'), ([9, 10], 'bright~blue'), ([11], 'hair'), ([12], 'and'), ([13], 'a'), ([14], 'Polish'), ([15], 'accent.')]
  if pmb_id == 'd0352_p06':
    return [([0, 1], 'The~longer'), ([2], 'you'), ([3], 'waste'), ([4], 'time'), ([5], 'in'), ([6], 'talk,'), ([7, 8], 'the~more'), ([9], 'certain'),
            ([10], 'it'), ([11], 'is'), ([12], 'that'), ([13], 'you'), ([14], 'will'), ([15], 'lose.')]
  if pmb_id == 'd0352_p10':
    return [([0], "Don't"), ([1], 'tell'), ([2], 'me'), ([3], 'any'), ([4], 'more'), ([5], 'about'), ([6], 'the'), ([7], 'crime,'),
            ([8, 9], 'in~fact,'), ([10], "don't"), ([11], 'tell'), ([12], 'me'), ([13], 'any'), ([14], 'more'), ([15, 16], 'full~stop.')]
  if pmb_id == 'd0352_p16':
    return [([0], 'He'), ([1], 'had'), ([2], 'lived'), ([3], 'with'), ([4], 'his'), ([5], 'mother'), ([6], 'till'), ([7], 'she'), ([8], 'died'),
            ([9], 'after'), ([10], 'a'), ([11], 'long'), ([12], 'and'), ([13], 'painful'), ([14], 'fight'), ([15], 'with'),
            ([16, 17], 'lung~cancer.')]
  if pmb_id == 'd0352_p30':
    return [([0], 'Colin'), ([1], 'and'), ([2], 'Julie'), ([3], 'walked'), ([4], 'across'), ([5], 'a'), ([6, 7], 'parade~ground'), ([8], 'past'),
            ([9], 'a'), ([10], 'lot'), ([11], 'of'), ([12], 'men'), ([13], 'walking'), ([14], 'around'), ([15], 'in'), ([16], 'uniform'),
            ([17], 'carrying'), ([18], 'guns.')]
  if pmb_id == 'd0352_p37':
    return [([0], 'Rachel'), ([1], 'for'), ([2], 'no'), ([3], 'reason'), ([4, 5], 'at~all'), ([6], 'dropped'), ([7], 'the'), ([8], 'fish'),
            ([9], 'slice'), ([10], 'on'), ([11], 'the'), ([12], 'floor'), ([13], 'and'), ([14], 'ran'), ([15], 'from'), ([16], 'the'), ([17], 'room'),
            ([18], 'crying.')]
  if pmb_id == 'd0352_p44':
    return [([0], 'Harvey'), ([1], 'longed'), ([2], 'for'), ([3], 'a'), ([4], 'son'), ([5], 'to'), ([6], 'carry'), ([7], 'on'), ([8], 'the'),
            ([9, 10], 'family~name'), ([11], 'and'), ([12], 'talked'), ([13], 'of'), ([14], 'little'), ([15], 'else'), ([16], 'in'), ([17], 'his'),
            ([18], 'letters'), ([19], 'home.')]
  if pmb_id == 'd0352_p50':
    return [([0], 'I'), ([1], 'considered'), ([2], 'it'), ([3], 'to'), ([4], 'be'), ([5], 'the'), ([6], 'minimum'), ([7], 'information'), ([8], 'I'),
            ([9], 'could'), ([10], 'give'), ([11], 'out'), ([12, 13, 14], 'in~order~to'), ([15], 'bring'), ([16], 'the'), ([17], 'situation'),
            ([18], 'under'), ([19], 'control.')]
  if pmb_id == 'd0352_p55':
    return [([0], 'To'), ([1], 'one'), ([2], 'side,'), ([3, 4, 5], 'out~of~sight'), ([6], 'of'), ([7], 'the'), ([8], 'priests'),
            ([9], 'on'), ([10], 'the'), ([11], 'steps'), ([12], 'of'), ([13], 'their'), ([14], 'temple,'), ([15], 'was'), ([16], 'a'),
            ([17], 'circle'), ([18], 'of'), ([19], 'stones.')]
  if pmb_id == 'd0352_p61':
    return [([0], 'Putting'), ([1], 'himself'), ([2], 'between'), ([3], 'Fred'), ([4], 'and'), ([5], 'his'), ([6], 'dad'), ([7], 'could'),
            ([8], 'only'), ([9], 'do'), ([10], 'more'), ([11], 'harm'), ([12], 'than'), ([13], 'good'), ([14, 15], 'at~least'), ([16], "that's"),
            ([17], 'what'), ([18], 'Neil'), ([19], 'told'), ([20], 'him.')]
  if pmb_id == 'd0352_p63':
    return [([0], 'By'), ([1], 'the'), ([2], 'time'), ([3], 'he'), ([4], 'got'), ([5], 'home'), ([6], 'he'), ([7], 'was'), ([8], 'feeling'),
            ([9], 'a'), ([10], 'little'), ([11, 12], 'light~headed'), ([13], 'and'), ([14], 'felt'), ([15], 'like'), ([16], 'he'), ([17], 'could'),
            ([18], 'take'), ([19], 'on'), ([20], 'the'), ([21], 'world.')]
  if pmb_id == 'd0352_p65':
    return [([0], 'Once'), ([1], "he'd"), ([2], 'chopped'), ([3], 'just'), ([4], 'about'), ([5], 'half,'), ([6], 'he'), ([7], 'stuck'), ([8], 'the'),
            ([9], 'axe'), ([10], 'into'), ([11], 'the'), ([12, 13], 'chopping~block'), ([14], 'and'), ([15], 'made'), ([16], 'his'), ([17], 'way'),
            ([18], 'inside'), ([19], 'to'), ([20], 'have'), ([21], 'breakfast.')]
  if pmb_id == 'd0352_p70':
    return [([0], 'Peter'), ([1], 'went'), ([2], 'to'), ([3], 'bed'), ([4], 'giving'), ([5], 'himself'), ([6], 'his'), ([7], 'word'), ([8], 'that'),
            ([9], 'while'), ([10], 'Hugh'), ([11], "wasn't"), ([12], 'around'), ([13], "he'd"), ([14], 'look'), ([15], 'out'), ([16], 'for'),
            ([17], 'George'), ([18, 19, 20], 'as~much~as'), ([21], 'he'), ([22], 'could.')]
  if pmb_id == 'd0352_p72':
    return [([0], 'Ever'), ([1], 'devoted'), ([2], 'to'), ([3], 'his'), ([4], 'wife'), ([5], 'he'), ([6], 'was'), ([7], 'at'), ([8], 'her'),
            ([9], 'side'), ([10], 'in'), ([11], 'the'), ([12], 'delivery'), ([13], 'room'), ([14], 'when'), ([15], 'she'), ([16, 17], 'gave~birth'),
            ([18], 'to'), ([19], 'a'), ([20], 'baby'), ([21], 'girl'), ([22], 'in'), ([23], '1978.')]
  if pmb_id == 'd0352_p75':
    return [([0], 'Phil'), ([1], 'ran'), ([2], 'up'), ([3], 'to'), ([4], 'the'), ([5, 6], 'water~tank,'), ([7], 'his'), ([8], 'mum'), ([9], 'would'),
            ([10], 'kill'), ([11], 'Nigel'), ([12], 'if'), ([13], 'she'), ([14], 'knew'), ([15], "he'd"), ([16], 'sent'), ([17], 'him'), ([18], 'up'),
            ([19], 'the'), ([20], 'hill'), ([21], 'without'), ([22], 'a'), ([23], 'jacket.')]
  if pmb_id == 'd0352_p78':
    return [([0], 'Mary'), ([1], 'had'), ([2], 'absolutely'), ([3], 'no'), ([4], 'idea'), ([5], 'what'), ([6], 'the'), ([7], 'woman'), ([8], 'was'),
            ([9], 'talking'), ([10], 'about'), ([11], 'but'), ([12], 'there'), ([13], 'were'), ([14], 'few'), ([15], 'things'), ([16], 'that'),
            ([17], 'annoyed'), ([18], 'her'), ([19, 20], 'more~than'), ([21], 'adults'), ([22], 'reading'), ([23], "children's"), ([24], 'books.')]
  if pmb_id == 'd0352_p82':
    return [([0], 'If'), ([1], 'you'), ([2], 'were'), ([3], 'coming'), ([4], 'from'), ([5], 'the'), ([6], 'high'), ([7], 'street'), ([8], 'you'),
            ([9], 'had'), ([10], 'to'), ([11], 'walk'), ([12], 'past'), ([13], 'three'), ([14], 'or'), ([15], 'four'), ([16], 'perfectly'),
            ([17], 'good'), ([18], 'pubs'), ([19], 'to'), ([20], 'get'), ([21], 'to'), ([22], 'The'), ([23, 24], 'Three~Cups.')]
  if pmb_id == 'd0352_p85':
    return [([0], 'There'), ([1], 'was'), ([2], 'no'), ([3], 'television'), ([4], 'in'), ([5], 'the'), ([6], 'lounge,'), ([7], 'there'), ([8], 'was'),
            ([9], 'a'), ([10], 'small'), ([11], 'one'), ([12], 'in'), ([13], 'the'), ([14], 'kitchen'), ([15], 'but'), ([16], 'in'), ([17], 'the'),
            ([18], 'lounge'), ([19], 'all'), ([20], 'seats'), ([21], 'faced'), ([22], 'the'), ([23, 24], 'record~player.')]
  if pmb_id == 'd0352_p90':
    return [([0], 'Jenny'), ([1], 'went'), ([2], 'to'), ([3], 'the'), ([4, 5], 'sitting~room'), ([6], 'while'), ([7], 'Ross'), ([8], 'finished'),
            ([9], 'off'), ([10], 'his'), ([11], 'tea'), ([12], 'and'), ([13], 'washed'), ([14], 'up'), ([15], 'his'), ([16], 'dishes'),
            ([17], 'then'), ([18], 'went'), ([19], 'for'), ([20], 'a'), ([21], 'bath'), ([22], 'and'), ([23], 'straight'), ([24], 'to'),
            ([25], 'bed.')]
  if pmb_id == 'd0353_p06':
    return [([0], 'Tom'), ([1], 'walked'), ([2], 'into'), ([3], 'the'), ([4], 'visiting'), ([5], 'room'), ([6], 'glad'), ([7], 'that'), ([8], 'his'),
            ([9], 'mum'), ([10], 'was'), ([11], 'making'), ([12], 'it'), ([13], 'back'), ([14], 'to'), ([15], 'see'), ([16], 'him'), ([17], 'so'),
            ([18], 'quickly'), ([19], 'or'), ([20, 21], 'at~least'), ([22], 'when'), ([23], 'they'), ([24], 'told'), ([25], 'him'), ([26], 'he'),
            ([27], 'had'), ([28], 'a'), ([29], 'visitor'), ([30], 'he'), ([31], 'assumed'), ([32], 'it'), ([33], 'was'), ([34], 'his'),
            ([35], 'mum.')]
  return [([idx], x) for idx, x in enumerate(sent.split(' '))]


base_folder = '/home/pramono/work/drs/local_neural_drs'
report_folder = to_abspath(base_folder, 'thesis')
data_folder = to_abspath(report_folder, 'data')

stimuli_folder = to_abspath(base_folder, 'thesis/data/frank_etal/')
stimuli_ref = parse_research_data(stimuli_folder)
manual_ref = load_indexed_manual_anaphoras()
pmb_id_ref = load_pmb_ids()

spr_data_with_subj = load_spr_rt_with_subj_by_keys()

et_data_with_subj = load_et_rt_with_subj_by_keys()
et_sent_nrs = load_et_with_subj_df()['sent_nr'].unique()

extracted_spr_sbn_data_with_subj = []
extracted_et_sbn_data_with_subj = []

sbn_drs_folder = to_abspath(base_folder, f'data/box_forms/frank-etal-2013/boxer_data_v3')
indexed_aligned_lines = []

skipped_pmb_ids = []
fullset_lambdas = dict()

for filename in sorted(os.listdir(sbn_drs_folder)):
  indexed_aligned_line = {}
  indexed_aligned_lines.append(indexed_aligned_line)
  src = to_abspath(sbn_drs_folder, filename)
  with open(src) as scraped_file:
    pmb_id = filename.replace(".txt", "")
    print('-' * 10)
    file_lines = scraped_file.readlines()
    sent = file_lines[0][2:].strip()
    stimulus_info = stimuli_ref[sent]
    sent_nr = stimulus_info['sent_nr']
    file_content = ''.join(file_lines)
    file_content = file_content.split('% Sequence Box Notation')
    data_to_process = file_content[1].split('% Incremental DRS')
    sbn_raw_text = data_to_process[0].strip()
    lambda_raw_text = data_to_process[1].strip()

    context_map, integration_map, sbn_lines_list, full_sbn_info_list = extract_sbn_integration(sent, sbn_raw_text)
    non_connector_sbn_lines = [x for x in full_sbn_info_list if x.sbn_word is not None]

    # check sbn parsing order. If the sentence is not in order. report this as misparsing and skip.
    sbn_words = [x.word_info.word_with_context.replace('--', '') for x in full_sbn_info_list if 'word_info' in x]
    sbn_parsing_in_order = sent.replace(' ', '') == re.sub(' +', '', ''.join(sbn_words))

    wrap_print(f'pmb_id: {pmb_id} sent_nr: {sent_nr}')
    if not sbn_parsing_in_order:
      skipped_pmb_ids.append(pmb_id)
      wrap_print(f'!! is not parsed in order of the sentence or has an issue with the anaphora. We skip this! See comparison:')
      wrap_print(f'sent = {sent.replace(" ", "")}')
      wrap_print(f'sbn  = {re.sub(" +", "", "".join(sbn_words))}')
    elif filename == 'd0352_p35.txt':
      skipped_pmb_ids.append(pmb_id)
      wrap_print(f'!! has cataphora marked as anaphora. We skip this!')
    else:
      enumerated_tokens = get_sentence_tokens(pmb_id, sent)
      lambda_lines = extract_lambda_info(sent_nr, enumerated_tokens, lambda_raw_text)

  fullset_lambdas[pmb_id] = lambda_lines


In [ ]:
def contains_prop_of(lline, prop_pred):
  return True in [x[1] == prop_pred for x in lline if x[0] == 'PROP']


def get_by_cat(lline, cat):
  return [x for x in lline if x[0] == cat]


def get_cat_counts(lline):
  groups = defaultdict(list)
  for idx, el in enumerate(lline):
    cat, pred, args = el
    groups[cat].append((idx, el))

  return sorted([(str([y[1] for y in values]) , len(values)) for key, values in groups.items()], key=lambda x: x[0], reverse=True)


prop_to_search = 'QuantityOf'
prop_counter = Counter()
data_counter = defaultdict(lambda: Counter())
for single_pmb_data in fullset_lambdas.values():
  for idx, sent_token, storage, left_over_elements, used_elements, prev_full_sbntokens, syncat, simplified_sem, calculated_change in single_pmb_data:
    lline = left_over_elements + used_elements
    nouns = '-'.join([x[1] for x in get_by_cat(lline, 'NOUN')])
    data_counter[syncat].update({
      f'{simplified_sem} <==> {get_cat_counts(lline)}': 1
    })
  # if contains_prop_of(lline, prop_to_search):
  #   nouns = '-'.join([f"{x[0]}:{x[1]}" for x in lline])
  #   # nouns = '-'.join([f"{x[0]}:{x[1]}{x[2]}" for x in lline])
  #   prop_counter.update({
  #     prop_to_search + '_' + nouns + '==' + sent_token.lower(): 1
  #   })


# print(set([x[len(prop_to_search) + 1:].split('=')[0] for x in prop_counter.keys()]))
# pprint(prop_counter)
def sum_valuescount(cdict):
  total = 0
  for key, val in cdict.items():
    total += val
  return total

min_to_print = 1
pprint(sorted([(key, sum_valuescount(values), sorted([(x, y) for x, y in values.items() if y >= min_to_print], key=lambda y: y[1], reverse=True)) for key, values in data_counter.items()], key=lambda x: x[1], reverse=True))
# pprint({key: dict(values) for key, values in data_counter.items()})



In [14]:
import pandas as pd
from script.sbnutils import determine_ori_sent_token_idxs_based_on_sbn
from script.utils import to_abspath

data_folder = '/home/pramono/work/drs/local_neural_drs/thesis/data/R_prepared'
# df = pd.read_csv(to_abspath(data_folder, 'R_et_rt_skipped_resolved_df.tsv'), sep='\t')
df = pd.read_csv(to_abspath(data_folder, 'R_storage_et_rt_skipped_update_storage_df.tsv'), sep='\t')
# df = pd.read_csv(to_abspath(data_folder, 'R_storage_et_spill_skipped_update_storage_df.tsv'), sep='\t')

df.shape
# df
# df[df.spill_fp >= 0]
# .groupby('spill_skipped')['spill_fp'].count()
# df.groupby('spill_skipped')['spill_fp'].count()


(15002, 21)